<a href="https://colab.research.google.com/github/PavanPasidu/ScriboSense/blob/saadha/wording_nb/wording2_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from sklearn.model_selection import train_test_split


In [3]:
import pandas as pd

dataset_path = '/content/drive/My Drive/Student Summarizer/dataset/'

# Read CSV file
summaries = pd.read_csv(dataset_path + 'summaries_train.csv')

# Display the first few rows of the dataset
print(summaries.head())


     student_id prompt_id                                               text  \
0  000e8c3c7ddb    814d6b  The third wave was an experimentto see how peo...   
1  0020ae56ffbf    ebad26  They would rub it up with soda to make the sme...   
2  004e978e639e    3b9047  In Egypt, there were many occupations and soci...   
3  005ab0199905    3b9047  The highest class was Pharaohs these people we...   
4  0070c9e7af47    814d6b  The Third Wave developed  rapidly because the ...   

    content   wording  
0  0.205683  0.380538  
1 -0.548304  0.506755  
2  3.128928  4.231226  
3 -0.210614 -0.471415  
4  3.272894  3.219757  


In [4]:
df=summaries
df.head()


,student_id,prompt_id,text,content,wording
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757


In [5]:
duplicate_rows = df[df.duplicated()]

# Display the duplicate rows
print("Duplicate Rows:" , str(len(duplicate_rows)))

Duplicate Rows: 0


In [6]:
columns_to_drop = ['student_id', 'prompt_id','content']
df = df.drop(columns=columns_to_drop)

df.head()

,text,wording
0,The third wave was an experimentto see how peo...,0.380538
1,They would rub it up with soda to make the sme...,0.506755
2,"In Egypt, there were many occupations and soci...",4.231226
3,The highest class was Pharaohs these people we...,-0.471415
4,The Third Wave developed rapidly because the ...,3.219757


In [7]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.8 MB/s eta 0:00:00


In [8]:
import torch
import sys
import numpy as np

from transformers import GPT2Tokenizer, GPT2LMHeadModel
# Load pre-trained model (weights)
with torch.no_grad():
        model = GPT2LMHeadModel.from_pretrained('gpt2')
        model.eval()
# Load pre-trained model tokenizer (vocabulary)
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

def score(sentence):
    tokenize_input = tokenizer.encode(sentence)
    tensor_input = torch.tensor([tokenize_input])
    loss=model(tensor_input, labels=tensor_input)[0]
    return np.exp(loss.detach().numpy())

# Test the score function
sentence = "The third wave was an experiment to see how people"
grammar_score = score(sentence.strip())
print("Grammar Score:", grammar_score)

Grammar Score: 46.55513


In [10]:
# Test the score function
sentence = "Some happy hvhvhg."
grammar_score = score(sentence.strip())
print("Grammar Score:", grammar_score)

Grammar Score: 317.88242


In [ ]:
grammar_scores = []

for sentence in df['text']:
    grammar_score = score(sentence)
    grammar_scores.append(grammar_score)

df['grammar_score'] = grammar_scores

In [ ]:
X = df[['grammar_score']]
y = df['wording']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

train_data = X_train.copy()
train_data['wording'] = y_train['wording']

sns.scatterplot(data=train_data, x='grammar_score', y='wording')
plt.show()

correlation = X_train['grammar_score'].corr(y_train['wording'])
print("Correlation between grammar score and wording:", correlation)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)